In [ ]:
import os
import tensorflow as tf
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
from tensorflow import keras
import cv2
import pickle
from tensorflow.keras import backend as K
from PIL import Image
import matplotlib.patches as mpatches

In [ ]:
PATH = '/kaggle/input/bridge-cracks-image/DeepPCB/PCBData'

In [ ]:
normal = []
defect = []
defectlog = []
path_2 = [os.path.join(PATH,dir) for dir in os.listdir(PATH) if '.' not in dir]
for p in tqdm(path_2,total=len(path_2)):
    path_3 = os.path.join(p,sorted(os.listdir(p))[0])
    normal +=[os.path.join(path_3,dir)for dir in os.listdir(path_3) if 'temp' in dir]
    defect +=[os.path.join(path_3,dir)for dir in os.listdir(path_3) if 'test' in dir]
    path_4 = os.path.join(p,sorted(os.listdir(p))[1])
    defectlog +=[os.path.join(path_4,dir)for dir in os.listdir(path_4)]
normal.sort()
defect.sort()
defectlog.sort()

In [ ]:
img0 = []
img1 = []
for img_path in tqdm(normal,total=len(normal)):
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img,(128,128))
    img0.append(img)
for img_path in tqdm(defect,total=len(defect)):
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img,(128,128))
    img1.append(img)
img0 = np.array(img0)
img1 = np.array(img1)

In [ ]:
box = []
i=0
for log in tqdm(defectlog,total=len(defectlog)):
    temp = pd.read_csv(log,sep=' ' ,names=['x1', 'y1', 'x2', 'y2', 'defect'], header=None).values.tolist()
    box.append(temp)
    i+=1
boxarr = []
for i in tqdm(range(len(box))):
    narr = np.zeros((640,640,1))
    for j in range(len(box[i])):
        x1,y1,x2,y2,d = box[i][j]
        narr = cv2.rectangle(narr,(x1,y1),(x2,y2),d,-1)
    narr = cv2.resize(narr,(88,88))
    boxarr.append(narr)
boxarr = np.array(boxarr)

In [ ]:
plt.imshow(np.reshape(boxarr[0],(88,88)),cmap='gnuplot')
plt.show()

In [ ]:
plt.imshow(np.reshape(img1[0],(128,128)),cmap='gnuplot')
plt.show()

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((img1,boxarr))

In [ ]:
n=1500
dataset = dataset.shuffle(n)
train_dataset = dataset.take(int(n*0.8)).batch(100)
test_dataset = dataset.skip(int(n*0.8)).batch(100)

In [ ]:
## layer5 cnn bn he flip
inputs = keras.layers.Input((128,128,1))
# x = keras.layers.experimental.preprocessing.Resizing(128,128)(inputs)
x = keras.layers.experimental.preprocessing.Rescaling(1./255)(inputs)
x = keras.layers.experimental.preprocessing.RandomFlip(mode='horizontal_and_vertical')(x)

x = keras.layers.Conv2D(filters=64,kernel_size=3,kernel_initializer=tf.keras.initializers.HeNormal())(x) #126 126 64
x = keras.layers.Activation('relu')(x)
x = keras.layers.Conv2D(filters=64,kernel_size=3,kernel_initializer=tf.keras.initializers.HeNormal())(x) #124 124 64
x = keras.layers.Activation('relu')(x)
x1 = tf.identity(x)
x1 = keras.layers.experimental.preprocessing.CenterCrop(92,92)(x1)

x = keras.layers.MaxPooling2D((2, 2), strides=2)(x) #62 62,64

x = keras.layers.Conv2D(filters=128,kernel_size=3,kernel_initializer=tf.keras.initializers.HeNormal())(x) #60 60 128
x = keras.layers.Activation('relu')(x)
x = keras.layers.Conv2D(filters=128,kernel_size=3,kernel_initializer=tf.keras.initializers.HeNormal())(x) #58 58 128
x = keras.layers.Activation('relu')(x)
x2 = tf.identity(x)
x2 = keras.layers.experimental.preprocessing.CenterCrop(50,50)(x2)

x = keras.layers.MaxPooling2D((2, 2), strides=2)(x) #29 29 128

x = keras.layers.Conv2D(filters=256,kernel_size=3,kernel_initializer=tf.keras.initializers.HeNormal())(x) #27 27 256
x = keras.layers.Activation('relu')(x)
x = keras.layers.Conv2D(filters=256,kernel_size=3,kernel_initializer=tf.keras.initializers.HeNormal())(x) #25 25 256
x = keras.layers.Activation('relu')(x)

x = tf.keras.layers.Conv2DTranspose(filters=128, kernel_size=2, strides=(2, 2),kernel_initializer=tf.keras.initializers.HeNormal())(x) # 50 50 128

x = keras.layers.Concatenate()([x,x2]) #50 50 256
x = keras.layers.Conv2D(filters=128,kernel_size=3,kernel_initializer=tf.keras.initializers.HeNormal())(x) #48 48 128
x = keras.layers.Activation('relu')(x)
x = keras.layers.Conv2D(filters=128,kernel_size=3,kernel_initializer=tf.keras.initializers.HeNormal())(x) #46 46 128
x = keras.layers.Activation('relu')(x)

x = tf.keras.layers.Conv2DTranspose(filters=64, kernel_size=2, strides=(2, 2),kernel_initializer=tf.keras.initializers.HeNormal())(x) # 92 92 64

x = keras.layers.Concatenate()([x,x1]) #92,92 128
x = keras.layers.Conv2D(filters=64,kernel_size=3,kernel_initializer=tf.keras.initializers.HeNormal())(x) #90 90 64
x = keras.layers.Activation('relu')(x)
x = keras.layers.Conv2D(filters=64,kernel_size=3,kernel_initializer=tf.keras.initializers.HeNormal())(x) #88 88 64
x = keras.layers.Activation('relu')(x)

outputs = keras.layers.Conv2D(filters=7,kernel_size=1,kernel_initializer=tf.keras.initializers.HeNormal())(x) #88 88 7

In [ ]:
model = keras.Model(inputs, outputs)
model.summary()

In [ ]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
)

In [ ]:
history = model.fit(train_dataset,epochs=500,validation_data=test_dataset,verbose=1)

In [ ]:
plt.plot(history.history['val_loss'])
plt.show()


In [ ]:
out = model.predict(tf.reshape(img1[0],(-1,128,128,1)))
out2 = np.argmax(out,axis=3)
plt.imshow(out2[0],cmap='gnuplot')

In [ ]:
img_arr_resized = cv2.resize(img1[0], (88, 88))
img_arr_resized = cv2.cvtColor(img_arr_resized,cv2.COLOR_GRAY2RGB)

In [ ]:
plt.imshow(img_arr_resized)

In [ ]:
pred = model.predict(tf.reshape(img1[0],(-1,128,128,1)))
pred = np.argmax(pred,axis=3)[0] 

In [ ]:
new_pred = np.zeros((88, 88, 3), dtype=np.uint8)

In [ ]:
new_pred[pred==0] = img_arr_resized[pred==0]
new_pred[pred==1] = [225, 228, 0] # open
new_pred[pred==2] = [1,0,255] # short
new_pred[pred==3] = [29, 219, 22] #mousebite
new_pred[pred==4] = [255, 0, 0] #spur
new_pred[pred==5] = [103, 0, 0] #copper
new_pred[pred==6] = [255, 0, 127] # pin-hole

In [ ]:
new_pred_img = Image.fromarray(new_pred)
plt.imshow(new_pred_img)

In [ ]:
img_arr_resized_img = Image.fromarray(img_arr_resized)
plt.imshow(img_arr_resized_img)

In [ ]:
blend_img1 = Image.fromarray(img_arr_resized)
blend_img2 = Image.fromarray(new_pred)
blend_img = Image.blend(blend_img1, blend_img2, 0.5)

In [ ]:
legend1 = mpatches.Patch(color='#FFE400', label='open')
legend2 = mpatches.Patch(color='#0100FF', label='short')
legend3 = mpatches.Patch(color='#1DDB16', label='mousebite')
legend4 = mpatches.Patch(color='#FF0000', label='spur')
legend5 = mpatches.Patch(color='#670000', label='copper')
legend6 = mpatches.Patch(color='#FF007F', label='pinhole')
plt.legend(loc='lower left', handles=[legend1, legend2, legend3, legend4, legend5, legend6], mode = 'expand', ncol = 3, fontsize=10)
plt.xticks([])
plt.yticks([])
plt.imshow(np.array(blend_img))
plt.show()